# 📈 Telecom Customer Churn — End‑to‑End ML Notebook
**Objective:** Predict churn and derive actionable retention strategies.

**Tools:** Python, pandas, scikit‑learn, matplotlib, (optional) SHAP / ELI5

> Put your dataset path in `DATA_PATH` below. Expected format: a CSV with a binary `churn` column (1 = churned, 0 = retained).

In [ ]:
# ==== Setup ====
import os, warnings, math, json
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (roc_auc_score, average_precision_score, precision_recall_curve,
                             roc_curve, classification_report, confusion_matrix)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

import joblib

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 120)
RANDOM_STATE = 42

# Try optional libraries quietly
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

try:
    import eli5
    from eli5.sklearn import PermutationImportance as ELI5PermutationImportance
    ELI5_AVAILABLE = True
except Exception:
    ELI5_AVAILABLE = False

print(f"SHAP available: {SHAP_AVAILABLE}") 
print(f"ELI5 available: {ELI5_AVAILABLE}") 


## 1) Load Data
Update `DATA_PATH` to your CSV. The CSV must include a binary target column **`churn`**.


In [ ]:
# ==== Data Loading ====
# TODO: Set your dataset path here
DATA_PATH = 'telecom_churn.csv'  # e.g., '/path/to/telecom_churn.csv'

assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}. Please update DATA_PATH above."
df = pd.read_csv(DATA_PATH)

# Normalize column names
df.columns = [c.strip().lower().replace(' ', '_') for c in df.columns]

# Basic checks
assert 'churn' in df.columns, "Your data must contain a binary 'churn' column (1 = churned, 0 = retained)."
print(df.shape)
df.head()

## 2) Quick EDA

In [ ]:
# Class balance
print('Class balance (churn=1 proportion):', df['churn'].mean())
print(df['churn'].value_counts(normalize=True))

# Missing rates
missing = df.isna().mean().sort_values(ascending=False)
print('Top missing columns:\n', missing.head(20))

# Simple numeric overview
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols]
print('Numeric columns:', num_cols[:20])
print('Categorical columns:', cat_cols[:20])

# Optional: a few basic histograms (matplotlib only; no seaborn per instructions)
for col in [c for c in num_cols if c != 'churn'][:6]:
    plt.figure()
    df[col].hist(bins=30)
    plt.title(f'Distribution: {col}')
    plt.xlabel(col); plt.ylabel('Count')
    plt.show()

## 3) Train/Validation Split

In [ ]:
X = df.drop(columns=['churn'])
y = df['churn'].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE
)

num_features = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_features = [c for c in X_train.columns if c not in num_features]

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, num_features),
        ('cat', categorical_pipeline, cat_features)
    ]
)

## 4) Train Models (Logistic Regression, Random Forest)

In [ ]:
# Logistic Regression
log_reg = Pipeline(steps=[('prep', preprocess),
                         ('clf', LogisticRegression(max_iter=200, class_weight='balanced', random_state=RANDOM_STATE))])
log_reg.fit(X_train, y_train)

# Random Forest
rf = Pipeline(steps=[('prep', preprocess),
                    ('clf', RandomForestClassifier(
                        n_estimators=400, max_depth=None, min_samples_split=2,
                        class_weight='balanced', random_state=RANDOM_STATE))])
rf.fit(X_train, y_train)

def evaluate(model, X_test, y_test, name='model'):
    proba = model.predict_proba(X_test)[:,1]
    auc = roc_auc_score(y_test, proba)
    ap = average_precision_score(y_test, proba)
    print(f"{name} | ROC-AUC: {auc:.4f} | PR-AUC: {ap:.4f}")
    fpr, tpr, _ = roc_curve(y_test, proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"{name} (AUC={auc:.3f})")
    plt.plot([0,1],[0,1],'--')
    plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curve'); plt.legend()
    plt.show()
    precision, recall, thresholds = precision_recall_curve(y_test, proba)
    plt.figure()
    plt.plot(recall, precision, label=f"{name} (AP={ap:.3f})")
    plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall Curve'); plt.legend()
    plt.show()
    return proba

print("\n=== Validation Metrics ===")
proba_lr = evaluate(log_reg, X_test, y_test, 'LogisticRegression')
proba_rf = evaluate(rf, X_test, y_test, 'RandomForest')

## 5) Business-Aware Threshold Tuning
Pick a probability threshold that balances precision/recall to your goals (e.g., retention budget).

In [ ]:
chosen_model = rf  # choose the better model after reviewing metrics above

proba = chosen_model.predict_proba(X_test)[:,1]
precision, recall, thresholds = precision_recall_curve(y_test, proba)

# Example: maximize F1
f1_scores = (2 * precision * recall) / (precision + recall + 1e-9)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[max(best_idx-1,0)]  # thresholds has len-1 compared to precision/recall
print(f"Best F1 threshold ≈ {best_threshold:.3f}; Precision={precision[best_idx]:.3f}, Recall={recall[best_idx]:.3f}")

plt.figure()
plt.plot(thresholds, f1_scores[1:], label='F1 vs Threshold')  # skip the first precision/recall point
plt.xlabel('Threshold'); plt.ylabel('F1 score'); plt.title('Threshold Tuning'); plt.legend()
plt.show()

## 6) Explainability (Permutation Importance, optional SHAP/ELI5)

In [ ]:
# Permutation Importance on validation set
from sklearn.utils.validation import check_is_fitted

# Get feature names after preprocessing
def get_feature_names(preprocess, num_features, cat_features):
    num_names = list(num_features)
    cat_encoder = preprocess.named_transformers_['cat'].named_steps['onehot']
    cat_names = list(cat_encoder.get_feature_names_out(cat_features)) if hasattr(cat_encoder, 'get_feature_names_out') else []
    return num_names + cat_names

# Build a clone that exposes the trained preprocess
prep = chosen_model.named_steps['prep']
clf = chosen_model.named_steps['clf']

X_test_transformed = prep.transform(X_test)
feature_names = get_feature_names(prep, num_features, cat_features)

perm = permutation_importance(clf, X_test_transformed, y_test, n_repeats=10, random_state=RANDOM_STATE, scoring='roc_auc')
imp = pd.DataFrame({'feature': feature_names, 'importance': perm.importances_mean}).sort_values('importance', ascending=False)
display(imp.head(30))

plt.figure()
imp.head(20).sort_values('importance').plot(kind='barh', x='feature', y='importance', legend=False)
plt.title('Permutation Importance (Top 20)')
plt.xlabel('Mean importance'); plt.tight_layout(); plt.show()

# Optional: SHAP (if available)
if SHAP_AVAILABLE:
    try:
        explainer = shap.Explainer(clf, X_test_transformed, feature_names=feature_names)
        shap_values = explainer(X_test_transformed[:200])  # sample for speed
        shap.summary_plot(shap_values, plot_type='bar', show=True)
        shap.summary_plot(shap_values, show=True)
    except Exception as e:
        print('SHAP failed:', e)

# Optional: ELI5 permutation importance (if available)
if ELI5_AVAILABLE:
    try:
        perm_eli5 = ELI5PermutationImportance(clf, random_state=RANDOM_STATE, scoring='roc_auc').fit(X_test_transformed, y_test)
        import eli5
        display(eli5.show_weights(perm_eli5, feature_names=feature_names, top=20))
    except Exception as e:
        print('ELI5 failed:', e)

## 7) Confusion Matrix at Chosen Threshold

In [ ]:
y_pred = (proba >= best_threshold).astype(int)
print(classification_report(y_test, y_pred, digits=3))
cm = confusion_matrix(y_test, y_pred)
print('Confusion Matrix:\n', cm)

## 8) Score Full Dataset & Create Segments
Segments:
- **At Risk**: predicted probability ≥ threshold  
- **Loyal**: probability < threshold and tenure/usage high (if available)  
- **Dormant**: low engagement/usage or long inactivity (if available)

Rules adjust automatically based on available columns.


In [ ]:
# Score entire dataset
proba_full = chosen_model.predict_proba(X)[:,1]
scored = X.copy()
scored['churn_prob'] = proba_full

# Heuristics for engagement/recency if columns exist
tenure_col = next((c for c in ['tenure','months_on_network','customer_age_months'] if c in scored.columns), None)
usage_cols = [c for c in scored.columns if any(k in c for k in ['mins','minutes','call','data_mb','data_gb','sms','recharge_amt','recharge_count']) and c != 'churn_prob']
recency_col = next((c for c in ['days_since_last_recharge','days_since_last_call','last_active_days'] if c in scored.columns), None)

# Compute simple engagement score if possible
engagement = None
if usage_cols:
    tmp = scored[usage_cols].copy()
    for c in usage_cols:
        if not np.issubdtype(tmp[c].dtype, np.number):
            tmp[c] = pd.to_numeric(tmp[c], errors='coerce')
    engagement = tmp.fillna(0).mean(axis=1)
else:
    engagement = pd.Series(np.nan, index=scored.index)

# Segment rules
def segment_row(row):
    p = row['churn_prob']
    high_risk = p >= best_threshold
    low_eng = False
    if not math.isnan(engagement.iloc[row.name]):
        low_eng = engagement.iloc[row.name] <= np.nanpercentile(engagement, 25)
    inactive = False
    if recency_col is not None and pd.notna(row.get(recency_col)):
        try:
            inactive = float(row.get(recency_col)) >= np.nanpercentile(scored[recency_col], 75)
        except Exception:
            inactive = False

    if high_risk:
        return 'At Risk'
    if inactive or low_eng:
        return 'Dormant'
    return 'Loyal'

scored['segment'] = [segment_row(scored.iloc[i]) for i in range(scored.shape[0])]

# Attach identifier if present
id_col = next((c for c in ['customer_id','user_id','msisdn','account_id','id'] if c in df.columns), None)
if id_col is not None:
    scored.insert(0, id_col, df[id_col])

# Save scored output
SCORED_PATH = 'scored_customers.csv'
scored.to_csv(SCORED_PATH, index=False)
print(f"Saved: {SCORED_PATH}")
scored.head()

## 9) Save Model Artifacts

In [ ]:
ARTIFACT_DIR = Path('artifacts'); ARTIFACT_DIR.mkdir(exist_ok=True)
joblib.dump(chosen_model, ARTIFACT_DIR / 'churn_model.joblib')
meta = {
    'created': str(pd.Timestamp.now()),
    'random_state': RANDOM_STATE,
    'best_threshold': float(best_threshold),
    'features_numeric': num_features,
    'features_categorical': cat_features
}
with open(ARTIFACT_DIR / 'metadata.json', 'w') as f:
    json.dump(meta, f, indent=2)
print('Artifacts saved in ./artifacts')

## 10) SQL Mini‑Guide (Aggregation Examples)

```sql
-- Calls & Usage
SELECT customer_id,
       SUM(outgoing_call_minutes) AS total_outgoing_mins,
       SUM(incoming_call_minutes) AS total_incoming_mins,
       SUM(data_mb) AS data_mb_total,
       COUNT(*) AS call_events
FROM cdr_events
GROUP BY customer_id;

-- Complaints
SELECT customer_id,
       COUNT(*) AS complaints_90d
FROM complaints
WHERE complaint_date >= CURRENT_DATE - INTERVAL '90 day'
GROUP BY customer_id;

-- Recharge frequency & amount
SELECT customer_id,
       COUNT(*) FILTER (WHERE recharge_date >= CURRENT_DATE - INTERVAL '30 day') AS recharge_cnt_30d,
       SUM(amount)  FILTER (WHERE recharge_date >= CURRENT_DATE - INTERVAL '30 day') AS recharge_amt_30d,
       MAX(recharge_date) AS last_recharge_date
FROM recharges
GROUP BY customer_id;

-- Join & label churn (example: no activity for 60+ days)
WITH base AS (... your aggregates ...)
SELECT b.*, CASE WHEN COALESCE(DATEDIFF('day', last_activity_date, CURRENT_DATE), 999) > 60 THEN 1 ELSE 0 END AS churn
FROM base b;
```
